In [ ]:
import numpy as np
import random
from collections import Counter
import math


In [ ]:
def measure_positional_diffusion(cipher_object, plaintext, num_trials=50):
    """
    Measures the positional predictability of changes in ciphertext.
    A lower score indicates more predictable changes (poor diffusion).
    """
    original_ciphertext, _ = cipher_object.encrypt(plaintext)
    cipher_len = len(original_ciphertext)
    
    changed_positions_all = []
    
    for _ in range(num_trials):
        # Create an incremental plaintext change
        modified_plaintext = list(plaintext)
        pos = random.randint(0, len(plaintext) - 1)
        modified_plaintext[pos] = random.choice(cipher_object.alphabet)
        modified_plaintext = "".join(modified_plaintext)
        
        try:
            modified_ciphertext, _ = cipher_object.encrypt(modified_plaintext)
        except Exception:
            continue
        
        changed_positions = [i for i, (c1, c2) in enumerate(zip(original_ciphertext, modified_ciphertext)) if c1 != c2]
        changed_positions_all.extend(changed_positions)
    
    if not changed_positions_all:
        return 0.0 # No changes were detected
    
    # Calculate a measure of distribution (e.g., variance of position indices)
    # A high variance indicates changes are scattered, low variance indicates clustering
    variance = np.var(changed_positions_all)
    
    # Normalize the score (you can adjust this formula)
    max_possible_variance = (cipher_len**2 - 1) / 12 # Variance of a uniform distribution
    normalized_score = variance / max_possible_variance
    
    return normalized_score

# Example Usage
# hill_cipher_obj = HillCipher(key="YOURKEY", n=3)
# score = measure_positional_diffusion(hill_cipher_obj, "ATTACKATDAWN")
# print(f"Positional Diffusion Score: {score:.2f}")

In [ ]:
def measure_value_confusion(cipher_object, plaintext, num_trials=50):
    """
    Measures the predictability of the character values that change.
    A lower score indicates more predictable changes (poor confusion).
    """
    original_ciphertext, _ = cipher_object.encrypt(plaintext)
    value_differences_all = []
    
    original_key = cipher_object.key
    
    for _ in range(num_trials):
        # Create an incremental key change
        modified_key = list(original_key)
        pos = random.randint(0, len(original_key) - 1)
        modified_key[pos] = random.choice(cipher_object.alphabet)
        modified_key = "".join(modified_key)
        
        try:
            modified_cipher_obj = HillCipher(key=modified_key, n=cipher_object.n)
            modified_ciphertext, _ = modified_cipher_obj.encrypt(plaintext)
        except (ValueError, IndexError):
            continue
            
        # Calculate the numerical difference for changed characters
        diffs = [
            abs(cipher_object.char_to_num[c1] - cipher_object.char_to_num[c2])
            for c1, c2 in zip(original_ciphertext, modified_ciphertext) if c1 != c2
        ]
        value_differences_all.extend(diffs)
    
    if not value_differences_all:
        return 0.0 # No changes were detected
        
    # Calculate a measure of uniformity or randomness, like normalized entropy
    # A high entropy indicates a wide, unpredictable range of changes
    counts = Counter(value_differences_all)
    probabilities = np.array(list(counts.values())) / len(value_differences_all)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    max_entropy = np.log2(len(set(value_differences_all)))
    if max_entropy == 0:
        return 0.0
        
    normalized_entropy = entropy / max_entropy
    
    return normalized_entropy

# Example Usage
# hill_cipher_obj = HillCipher(key="YOURKEY", n=3)
# score = measure_value_confusion(hill_cipher_obj, "QUANTIFYCONFUSION")
# print(f"Character Value Confusion Score: {score:.2f}")

In [ ]:
def measure_diffusion(cipher_object, plaintext, num_trials=50):
    """
    Measures diffusion by analyzing the randomness of change positions and
    the spread of character value changes.
    Returns two scores: positional randomness and character value spread.
    Scores range from 0 (predictable) to 1 (random).
    """
    original_ciphertext, _ = cipher_object.encrypt(plaintext)
    
    all_changed_positions = []
    all_value_differences = []
    
    for _ in range(num_trials):
        # Create an incremental plaintext change
        modified_plaintext = list(plaintext)
        pos = random.randint(0, len(plaintext) - 1)
        modified_plaintext[pos] = random.choice(cipher_object.alphabet)
        modified_plaintext = "".join(modified_plaintext)
        
        try:
            modified_ciphertext, _ = cipher_object.encrypt(modified_plaintext)
        except Exception:
            continue
        
        # Collect data on changes
        changed_positions = []
        value_differences = []
        
        for i, (c1, c2) in enumerate(zip(original_ciphertext, modified_ciphertext)):
            if c1 != c2:
                changed_positions.append(i)
                diff = abs(cipher_object.char_to_num[c1] - cipher_object.char_to_num[c2])
                value_differences.append(diff)

        all_changed_positions.extend(changed_positions)
        all_value_differences.extend(value_differences)
        
    if not all_changed_positions:
        return 0.0, 0.0 # No changes were detected
    
    # Calculate Positional Randomness (Entropy)
    pos_counts = Counter(all_changed_positions)
    pos_probabilities = np.array(list(pos_counts.values())) / len(all_changed_positions)
    pos_entropy = -np.sum(pos_probabilities * np.log2(pos_probabilities))
    max_pos_entropy = np.log2(len(set(all_changed_positions)))
    positional_score = pos_entropy / max_pos_entropy if max_pos_entropy > 0 else 0.0
    
    # Calculate Character Value Spread (Entropy)
    val_counts = Counter(all_value_differences)
    val_probabilities = np.array(list(val_counts.values())) / len(all_value_differences)
    val_entropy = -np.sum(val_probabilities * np.log2(val_probabilities))
    max_val_entropy = np.log2(len(set(all_value_differences)))
    value_score = val_entropy / max_val_entropy if max_val_entropy > 0 else 0.0
    
    return positional_score, value_score

# Example Usage
# hill_cipher_obj = HillCipher(key="HILLCIPHERKEY", n=3)
# pos_score, val_score = measure_diffusion(hill_cipher_obj, "QUANTIFYCONFUSIONANDDIFFUSION")
# print(f"Positional Randomness Score (Diffusion): {pos_score:.2f}")
# print(f"Character Value Spread Score (Diffusion): {val_score:.2f}")